# **TELCO CHURN PREDICTION**

## **A. All Packages be used on this project**

In [11]:
# packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## **B. DATASET**

In [6]:
# load dataset
from dotenv import load_dotenv
import os
load_dotenv()  

# churn_dataset
churn_dataset = os.getenv("churn_dataset_path")
df_churn = pd.read_excel(churn_dataset, sheet_name="Sheet1")

# LIS_dataset
lis_dataset = os.getenv("lis_dataset_path")
df_lis = pd.read_excel(lis_dataset, sheet_name="lis_telco")


## **C. DATA PRE-PROCESSING**

### **C.1 CHURN DATASET**

#### **C.1.1 DATA CLEANING**

Pada Section ini kami memilih untuk membuang column yang tidak relevan dengan churn analisis.


In [ ]:
raw_churn = df_churn.copy()

# drop columns that are not relevant to churn analysis
raw_churn = raw_churn.drop(columns=['DIVISI','HSI_B2B','ND_HSI_IPTV','NCLI'],axis=1)

#### **C.1.2 DATA FORMATTING**

pada section ini, kita akan mengubah data type yang keliru diformat dan ditransform secara default. adapun data yang akan diformat terdapat pada kolom : 
  * Formatting : [TGL_PSB] dan [TGL_PS] masih bertipe numeric, sehingga perlu di transformasi kedalam tipe date
  * Transforming : [UBIS] dengan nilai Business menjadi -> RBS

In [ ]:
# Date Handling

# 1. convert raw_churn[['TGL_PSB','TGL_PS']] to datetime
raw_churn['TGL_PSB'] = pd.to_datetime(raw_churn['TGL_PSB'], format='%Y%m%d')
raw_churn['TGL_PS'] = pd.to_datetime(raw_churn['TGL_PS'], format='%Y%m%d')

# 2. Transform [DIVISI] == 'Business' as 'RBS'
raw_churn['UBIS'] = raw_churn.loc[raw_churn['UBIS'] == 'Business', 'DIVISI'] = 'RBS'

In [88]:
raw_churn['CITEM'].value_counts()

CITEM
WM_20M        2566
HSIEF20M       975
HSIEF30M       963
HSIEF50M       684
HSIE50M        513
              ... 
INETR3M          1
INETG1M          1
INETFNL30M       1
INETR100M3       1
INET_1MB_H       1
Name: count, Length: 110, dtype: int64

#### **C.1.3 DATA CLASSIFICATION**

* Mapping kategori Length of Stay pada  df_churn ['LENGTH OF STAY CAT'] adalah sebagai berikut : 
  - < 6 Bulan = 1-6 months
  - 6 - 12 Bulan = 7-12 months 
  - 1-2 Tahun = 13-24 months
  - 2-3 Tahun = 25-36 months
  - 3-4 Tahun = 37-48 months
  - 4-5 Tahun = 49-60 months
  - &gt; 5 tahun = > 60 months


In [ ]:
# expected output : map all not null value in TGL_PSB and TGL_PS to "Length of Stay Cat"
# make a new column MONTH_DIFF
raw_churn['MONTH_DIFF'] = ((raw_churn['TGL_PS'].dt.year - raw_churn['TGL_PSB'].dt.year) * 12) + (raw_churn['TGL_PS'].dt.month - raw_churn['TGL_PSB'].dt.month)

# do mapping -> see the map_lenth_of_stay function on python_function folder
import sys
sys.path.append('../python_function') 
from length_of_stay_map import map_length_of_stay

raw_churn['LENGTH OF STAY CAT'] = map_length_of_stay(raw_churn['MONTH_DIFF'])

# check the result
print(raw_churn.loc[df_churn['LENGTH OF STAY CAT']=='Unidentified',['TGL_PSB','TGL_PS']])

         TGL_PSB     TGL_PS
1079  2023-03-03 2025-02-20
1472  2023-03-06 2025-02-23
2172  2022-08-19 2024-08-08
2309         NaT 2023-02-02
2329         NaT 2024-02-13
...          ...        ...
12378        NaT 2023-07-10
12448        NaT 2023-01-03
12475        NaT 2025-04-11
12553        NaT 2024-01-30
12636        NaT 2023-01-26

[155 rows x 2 columns]


### **C.2 LIS DATASET**

#### **C.2.1 DATA CLEANING**

Pada Section ini kami memilih untuk membuang column yang tidak relevan dengan churn analisis. selain itu, kami juga melakukan dropping segment pelanggan yang bukan RBS ([UNIT2] != 'RBS')

In [ ]:
raw_lis = df_lis.copy()

# 1. drop unecessary columns
raw_lis = raw_lis.drop(columns=['KW_IH','AMBANG','NCLI','CITEM','PRODTYPE','IS_INDIHOME','CEK_WICO','PERIOD','PERIOD_PS','UNIT','CEK_LGEST','INET_BASIC','LGEST','PACK_NAME','PLBLCL','Rev (Rp M)','TREMS_REV_P','ADDON','ALAMAT','AMBANG2','ASSET_ROWID','BA_ROWID','BLN_TGK','CA_ROWID','Calculation2','CEK_CGEST','CEK_2P','CEK_3P','CEK_HSIE','CEK_HSSP','CEK_HSSP (copy)','CEK_LOY','CEK_P_HSI','CEK_TO','CGEST','CITEM_EXT','EKOSISTEM','EMAIL','F_PRIORITY','F_PRIORITY_DESC','GROUP_HSIE','GROUP_PRO_HW','GRP_PRODUK','HARGA_HSI','HARGA_ONT','HSIE RECOM','HSSP RECOM','IHOME_INV','INET_OTHERS','IPSTATIC','JKW1','JKW2','JKW3','JKW4','KET_MIGRASI','KET_RESL','KET_SELLER','KW_IH2','KW_POTS','KW_UM_TXT','KW_UMUR','L_AMBANG','L_KW_UMUR','LINECATS_ITEM_ID','LINKAR','LOY_PROGRAM','NAMA','NAMA_AM','ND_REFERENCE','NDOS','NIK_AM','NIPNAS','NO_HP','P_DIGITAL','PKW1','PKW2','PKW3','PKW4','PRIO_MIG','PRIORITAS_MIG','PRIORITY','PRODUK2','REALM','RECOM_DIGI','REV (Jt)','REV INET(M)','REV VOICE (M)','REV_TXT','REV(M)','REVENUE_ADDON_INET','REVENUE_ADDON_TV','REVENUE_INET','REVENUE_TV','RID','ROOT_ASSET_ID','SELISIH','SELISIH2','SPEED','TAG_OTHER','TAHUN_PS','TECHNO','titik','TOT_TGK','TREMS_REV','TREMS_REV_REF','UNIT (copy)','UMUR PLG (Th)'],axis=1)


# 2. drop rows with not RBS values in ['UNIT2']
raw_lis =raw_lis[raw_lis['UNIT2'] == "RBS"]

#### **C.2.2 MISSING VALUES HANDLING**

pada section ini, missing values pada kolom berikut : ['ADDON_PRICE','ADDON_TOTAL','IS_IPTV','IS_POTS'] akan kita ubah menjadi 0 

In [98]:
# missing values handling

# 1. missing values in ['ADDON_PRICE'] will be replaced with 0
raw_lis['ADDON_PRICE'] = raw_lis['ADDON_PRICE'].fillna(0)

# 2. missing values in ['ADDON_TOTAL'] will be replaced with 0
raw_lis['ADDON_TOTAL'] = raw_lis['ADDON_TOTAL'].fillna(0)

# 3. missing values in ['IS_IPTV'] will be replaced with 0
raw_lis['IS_IPTV'] = raw_lis['IS_IPTV'].fillna(0)

# 4. missing values in ['IS_POTS'] will be replaced with 0
raw_lis['IS_POTS'] = raw_lis['IS_POTS'].fillna(0)


#### **C.2.3 DATA FORMATTING & TRANSFORMING**

pada section ini, kita akan mengubah data type yang keliru diformat dan ditransform secara default. adapun data yang akan diformat terdapat pada kolom : 
  - Formatting : [TGL_PS] -> should be formatted as date
  - Formatting : [NOTEL] -> should be formatted as int64
  - Transforming : [BW] -> BW_NULL should be stored as 0 then format data type as int64

In [ ]:
# Remove '_DEL' substring from NOTEL values
raw_lis['NOTEL'] = raw_lis['NOTEL'].astype(str).str.replace('_DEL', '', regex=False)

# Convert NOTEL to numeric (Int64)
raw_lis['NOTEL'] = pd.to_numeric(raw_lis['NOTEL'], errors='coerce').astype('Int64')

In [101]:
# data formatting

# 1. convert 'TGL_PS' from object to date
raw_lis['TGL_PS'] = pd.to_datetime(raw_lis['TGL_PS'], format='%m/%d/%Y %I:%M:%S %p')

# 2.1 Remove '_DEL' substring from NOTEL values
raw_lis['NOTEL'] = raw_lis['NOTEL'].astype(str).str.replace('_DEL', '', regex=False)
# 2.2 Convert NOTEL to numeric (Int64)
raw_lis['NOTEL'] = pd.to_numeric(raw_lis['NOTEL'], errors='coerce').astype('Int64')

# 3. transform ['Speed (MBps)'] == 'BW_NULL' as 0
raw_lis['Speed (MBps)'] = raw_lis.loc[raw_lis['Speed (MBps)'] == 'BW_NULL', 'Speed (MBps)'] = 0


#### **C.2.4 HANDLING DUPLICATE VALUES IN PRIMARY KEY**

kami memilih untuk melakukan drop pada duplicate value di primary key ['NOTEL'] dengan melakukan kriteria data yang di drop adalah data ['NOTEL'] dengan tanggal PS terlama.

In [117]:
# drop duplicate rows based on NOTEL and keep only the row with the smallest value in ['UMUR_PSB']

raw_lis = raw_lis.sort_values('UMUR_PSB').drop_duplicates(subset=['NOTEL'], keep='first')